In [ ]:
import pandas as pd
import logging 
import glob
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 500)
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [2]:
!ls -la

total 6701168
drwxr-xr-x  20 i851894  1694527156         640 Sep 13 05:37 .
drwxr-xr-x  23 i851894  1694527156         736 Sep 12 21:14 ..
drwxr-xr-x   3 i851894  1694527156          96 Sep 12 21:40 .ipynb_checkpoints
-rw-r--r--   1 i851894  1694527156       21404 Sep 12 22:02 Training.ipynb
drwxr-xr-x   3 i851894  1694527156          96 Sep 12 21:40 __pycache__
-rw-r--r--   1 i851894  1694527156      483007 Sep 12 21:52 body_pp.dpkl
-rw-r--r--@  1 i851894  1694527156  2852401417 Jan 17  2018 github_issues.csv
-rw-r--r--   1 i851894  1694527156    72133256 Sep 12 21:54 seq2seq_model_tutorial.h5
-rw-r--r--   1 i851894  1694527156       14145 Sep 12 21:38 seq2seq_utils.py
-rw-r--r--   1 i851894  1694527156      172604 Sep 12 21:52 title_pp.dpkl
-rw-r--r--   1 i851894  1694527156      504128 Sep 12 21:52 train_body_vecs.npy
-rw-r--r--   1 i851894  1694527156       86528 Sep 12 21:52 train_title_vecs.npy
-rw-r--r--   1 i851894  1694527156    72133256 Sep 12 21:53 tutorial_seq2seq.epoch01-v

## Process Data
Spli data into train and test set

In [3]:
data_file = 'github_issues.csv'

use_sample_data=False

if use_sample_data:
    training_data_size=2000
    traindf, testdf = train_test_split(pd.read_csv(data_file).sample(n=training_data_size), 
                                   test_size=.10)
else:
    traindf, testdf = train_test_split(pd.read_csv(data_file),test_size=.10)

#print out stats about shape of data
print(f'Train: {traindf.shape[0]:,} rows {traindf.shape[1]:,} columns')
print(f'Test: {testdf.shape[0]:,} rows {testdf.shape[1]:,} columns')

# preview data
traindf.head(3)    

Train: 4,798,937 rows 3 columns
Test: 533,216 rows 3 columns


,issue_url,issue_title,body
1063994,"""https://github.com/NutritionMate/Nutrition.Information.Api/issues/90""","as an administrator of the nutrition information api repository, i want new issues to be automatically populated with the definition of ready template, so that it is never accidentally forgotten.","as an administrator of the nutrition information api repository, i want new issues to be automatically populated with the definition of ready template, so that it is never accidentally forgotten. given a new issue, then automatically populate the description with the definition of ready template. do this by using https://github.com/blog/2111-issue-and-pull-request-templates"
5277470,"""https://github.com/ITS-UofIowa/its.uiowa.edu/issues/61""",change lynda.com links header to lynda video tutorials on info centers,"if lynda video links are added to an info center, lynda.com links is the header displayed above the links. can this be changed to lynda vido tutorials ? i think the latter is a better descriptor."
1564648,"""https://github.com/ValerioLyndon/MAL-Public-List-Designs/issues/13""",clarity - priority location and functionality is kind of a nuisance right now.,the hovering to display is annoying and could be improved. ! 2017-08-23 - 22 52 20 https://user-images.githubusercontent.com/29792052/29651456-b7093836-8855-11e7-9cb1-9c6dd5b638a4.png


In [4]:
train_body_raw = traindf.body.tolist()
train_title_raw = traindf.issue_title.tolist()
#preview output of first element
train_body_raw[0]

'as an administrator of the nutrition information api repository, i want new issues to be automatically populated with the definition of ready template, so that it is never accidentally forgotten. given a new issue, then automatically populate the description with the definition of ready template. do this by using https://github.com/blog/2111-issue-and-pull-request-templates'

In [5]:
%reload_ext autoreload
%autoreload 2
from ktext.preprocess import processor

/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
%%time
# Clean, tokenize, and apply padding / truncating such that each document length = 70
#  also, retain only the top 8,000 words in the vocabulary and set the remaining words
#  to 1 which will become common index for rare words 
body_pp = processor(keep_n=8000, padding_maxlen=70)
train_body_vecs = body_pp.fit_transform(train_body_raw)

Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/ktext/preprocess.py", line 90, in process_text
    return [tokenizer(cleaner(doc)) for doc in text]
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/ktext/preprocess.py", line 90, in <listcomp>
    return [tokenizer(cleaner(doc)) for doc in text]
  File "/User

  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/ktext/preprocess.py", line 57, in textacy_cleaner
    no_accents=True)
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/textacy/preprocess.py", line 217, in preprocess_text
    text = fix_bad_unicode(text, normalization='NFC')
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/textacy/preprocess.py", line 38, in fix_bad_unicode
    return fix_text(text, normalization=normalization)
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/ftfy/__init__.py", line 183, in fix_text
    normalization=normalization
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/ftfy/__init__.py", line 273, in fix_text_segment
    text = fixes.fix_encoding(text)
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/ftfy/fixes.py", line 110, in fix_encoding
    text, _ = fix_encoding_and_explain(text)
  File "/Users/i851894/anaconda/envs/py36/lib/pyth

  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/i851894/anaconda/envs/py36/lib/python3.6/site-packages/multiprocess/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/

KeyboardInterrupt: 

### Look at one example of processed issue bodies

In [ ]:
print('\noriginal string:\n', train_body_raw[0], '\n')
print('after pre-processing:\n', train_body_vecs[0], '\n')

In [ ]:
# Instantiate a text processor for the titles, with some different parameters
#  append_indicators = True appends the tokens '_start_' and '_end_' to each
#                      document
#  padding = 'post' means that zero padding is appended to the end of the 
#             of the document (as opposed to the default which is 'pre')
title_pp = processor(append_indicators=True, keep_n=4500, 
                     padding_maxlen=12, padding ='post')

# process the title data
train_title_vecs = title_pp.fit_transform(train_title_raw)

Look at one example of processed issue titles

In [ ]:
print('\noriginal string:\n', train_title_raw[0])
print('after pre-processing:\n', train_title_vecs[0])

Pickle this vectors for later use

In [ ]:
import dill as dpickle
import numpy as np

# Save the preprocessor
with open('body_pp.dpkl', 'wb') as f:
    dpickle.dump(body_pp, f)

with open('title_pp.dpkl', 'wb') as f:
    dpickle.dump(title_pp, f)

# Save the processed data
np.save('train_title_vecs.npy', train_title_vecs)
np.save('train_body_vecs.npy', train_body_vecs)

# Define Model
Load the data from disk

In [ ]:
from seq2seq_utils import load_decoder_inputs, load_encoder_inputs, load_text_processor


In [ ]:
encoder_input_data, doc_length = load_encoder_inputs('train_body_vecs.npy')
decoder_input_data, decoder_target_data = load_decoder_inputs('train_title_vecs.npy')

In [ ]:
num_encoder_tokens, body_pp = load_text_processor('body_pp.dpkl')
num_decoder_tokens, title_pp = load_text_processor('title_pp.dpkl')

#### Define Model Architecture

In [ ]:
%matplotlib  inline
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional, BatchNormalization
from keras import optimizers

In [ ]:
#arbitrarly set latent dimension for embedding and hidden units
latent_dim = 300

##### Define Model Architecture ######

########################
#### Encoder Model ####
encoder_inputs = Input(shape=(doc_length,), name='Encoder-Input')

# Word embeding for encoder (ex: Issue Body)
x = Embedding(num_encoder_tokens, latent_dim, name='Body-Word-Embedding', mask_zero=False)(encoder_inputs)
x = BatchNormalization(name='Encoder-Batchnorm-1')(x)

# We do not need the `encoder_output` just the hidden state.
_, state_h = GRU(latent_dim, return_state=True, name='Encoder-Last-GRU')(x)

# Encapsulate the encoder as a separate entity so we can just 
#  encode without decoding if we want to.
encoder_model = Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')

seq2seq_encoder_out = encoder_model(encoder_inputs)

########################
#### Decoder Model ####
decoder_inputs = Input(shape=(None,), name='Decoder-Input')  # for teacher forcing

# Word Embedding For Decoder (ex: Issue Titles)
dec_emb = Embedding(num_decoder_tokens, latent_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
dec_bn = BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)

# Set up the decoder, using `decoder_state_input` as initial state.
decoder_gru = GRU(latent_dim, return_state=True, return_sequences=True, name='Decoder-GRU')
decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out)
x = BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)

# Dense layer for prediction
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Final-Output-Dense')
decoder_outputs = decoder_dense(x)

########################
#### Seq2Seq Model ####

#seq2seq_decoder_out = decoder_model([decoder_inputs, seq2seq_encoder_out])
seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.001), loss='sparse_categorical_crossentropy')

### Examine Model 

In [ ]:
from seq2seq_utils import viz_model_architecture
seq2seq_Model.summary()
viz_model_architecture(seq2seq_Model)

## Train Model

In [ ]:
from keras.callbacks import CSVLogger, ModelCheckpoint

script_name_base = 'tutorial_seq2seq'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                   save_best_only=True)

batch_size = 1200
epochs = 7
history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.12, callbacks=[csv_logger, model_checkpoint])

In [ ]:
#save model
seq2seq_Model.save('seq2seq_model_tutorial.h5')

### See Example Results on Holdout set

In [ ]:
from seq2seq_utils import Seq2Seq_Inference
seq2seq_inf = Seq2Seq_Inference(encoder_preprocessor=body_pp,
                                 decoder_preprocessor=title_pp,
                                 seq2seq_model=seq2seq_Model)


In [ ]:
# this method displays the predictions on random rows of the holdout set
seq2seq_inf.demo_model_predictions(n=50, issue_df=testdf)

# Evaluate Model : BLEU Score

In [ ]:
#convenience function that generates predictions on holdout set and calculates BLEU Score

bleu_score = seq2seq_inf.evaluate_model(holdout_bodies=testdf.body.tolist(), 
                                        holdout_titles=testdf.issue_title.tolist())

In [ ]:
print(f'BLEU Score (avg of BLUE 1-4) on Holdout Set: {bleu_score * 100}')